In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None으로 지정할 경우 해당 컴퓨팅 유닛에 알맞은 dtype으로 저장됩니다. Tesla T4와 V100의 경우에는 Float16, Ampere+ 이상의 경우에는 Bfloat16으로 설정됩니다.
load_in_4bit = True # 메모리 사용량을 줄이기 위해서는 4bit 양자화를 사용하실 것을 권장합니다.

# 모델 및 토크나이저 선언
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # gated model을 사용할 경우 허깅페이스 토큰을 입력해주시길 바라겠습니다.
)
# LoRA Adapter 선언
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 0을 넘는 숫자를 선택하세요. 8, 16, 32, 64, 128이 추천됩니다.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # target module도 적절하게 조정할 수 있습니다.
    lora_alpha = 16,
    lora_dropout = 0, # 어떤 값이든 사용될 수 있지만, 0으로 최적화되어 있습니다.
    bias = "none",    # 어떤 값이든 사용될 수 있지만, "none"으로 최적화되어 있습니다.
    use_gradient_checkpointing = "unsloth", # 매우 긴 context에 대해 True 또는 "unsloth"를 사용하십시오.
    random_state = 42,
    use_rslora = False,
    loftq_config = None
)
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["text"]
    outputs = examples["text"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Linq-AI-Research/FinanceRAG",'FinQABench', split = "corpus")
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # True로 설정하면 짧은 텍스트 데이터에 대해서는 더 빠른 학습 속도로를 보여줍니다.
    args = TrainingArguments( # TrainingArguments는 자신의 학습 환경과 기호에 따라 적절하게 설정하면 됩니다.
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt_format.format(
        "다음 중 화폐의 시간가치에 관한 설명으로 옳지 않은 것은 무엇인가? A. 월 복리의 경우, 매월 적용되는 이자율은 연간 명목 이자율을 1/12로 나누어 산출한다. B. 투자 원금 및 기타 조건이 동일할 경우, 단리 방식보다 복리 방식에서 발생하는 이자가 더 크다. C. 일시불로 지급될 금액의 현재 가치는 미래 가치를 일정 기간 동안 할인율을 적용해 산출할 수 있다. D. 1,000,000원을 연 5% 복리로 2년 동안 예치했을 경우, 만기에 받을 세전 이자는 100,000원이다..", # instruction
        "", # output 생성을 위해 빈 칸으로 설정
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Qwen2 patching. Transformers = 4.45.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe MIG 3g.40gb. Max memory: 39.5 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth 2024.10.3 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.
Map (num_proc=2): 100%|██████████| 92/92 [00:00<00:00, 97.10 examples/s] 
max_steps is given, it will override any value given in num_train_epochs


['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n다음 중 화폐의 시간가치에 관한 설명으로 옳지 않은 것은 무엇인가? A. 월 복리의 경우, 매월 적용되는 이자율은 연간 명목 이자율을 1/12로 나누어 산출한다. B. 투자 원금 및 기타 조건이 동일할 경우, 단리 방식보다 복리 방식에서 발생하는 이자가 더 크다. C. 일시불로 지급될 금액의 현재 가치는 미래 가치를 일정 기간 동안 할인율을 적용해 산출할 수 있다. D. 1,000,000원을 연 5% 복리로 2년 동안 예치했을 경우, 만기에 받을 세전 이자는 100,000원이다..\n\n### Response:\nD. 1,000,000원을 연 5% 복리로 2년 동안 예치했을 경우, 만기에 받을 세전 이자는 100,000원이다.. \n\nThis statement is incorrect because if you invest 1,000,000원 at a 5% annual compound interest rate for 2 years, the total interest earned before taxes would be more than 100,000원. The formula to calculate compound interest is: Principal * (1 + Interest Rate)^Years - Principal. So, the total amount after 2 years would be 1,000,000원 * (1 + 0.05)^2 -  1,000,000원 =  1,102,500원. Thus, the interest earned would be  1,102,500원 -  1,000,000원 =  1,02,500원, which i